<a href="https://colab.research.google.com/github/jmohsbeck1/jpmc_mle/blob/final_project/DataDazzlers_DineOracle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Machine Learning Final Project

Team: DataDazzlers

Dataset: Yelp

In [2]:
import os
import warnings
import itertools
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
warnings.filterwarnings('ignore')

In [3]:
#Dataframe and numerical library
import pandas as pd 
import numpy as np
from scipy import stats
import json

#Visualization
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# %matplotlib inline
from matplotlib import rcParams

#Stats
from scipy import stats

#Machine Learning Model
#Metrics
from sklearn.metrics import mutual_info_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error


#Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

#Preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

#Linear Model
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge

#Ensemble
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

#Others
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier



#Hyper parameter
from sklearn import neighbors, datasets, model_selection

# Tensorflow
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

Load: yelp_academic_dataset_business_subset json file

/content/yelp_academic_dataset_business_subset.json

/content/yelp_academic_dataset_business_subset_II.json

In [4]:
df = pd.read_json('./yelp_academic_dataset_business_subset_II.json', lines=True, orient='records')


In [5]:
df.shape

(10, 14)

In [6]:
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [8]:
df.describe()

,postal_code,latitude,longitude,stars,review_count,is_open
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,51329.200000,36.293187,-90.877621,3.250000,19.000000,0.700000
std,26257.596314,3.919236,14.199812,1.086534,22.130924,0.483046
min,18054.000000,27.766590,-119.711197,1.500000,5.000000,0.000000
25%,34534.000000,34.872035,-90.339272,2.625000,7.750000,0.250000
50%,50165.000000,37.410359,-88.690015,3.250000,13.000000,1.000000
75%,63138.750000,38.612062,-83.741780,3.875000,18.000000,1.000000
max,93101.000000,40.338183,-75.155564,5.000000,80.000000,1.000000


In [9]:
df.corr()

,postal_code,latitude,longitude,stars,review_count,is_open
postal_code,1.000000,-0.256646,-0.938207,0.153362,-0.329294,-0.770437
latitude,-0.256646,1.000000,0.377305,-0.034377,0.392922,0.215070
longitude,-0.938207,0.377305,1.000000,-0.252178,0.334675,0.782083
stars,0.153362,-0.034377,-0.252178,1.000000,0.240280,-0.370479
review_count,-0.329294,0.392922,0.334675,0.240280,1.000000,0.093543
is_open,-0.770437,0.215070,0.782083,-0.370479,0.093543,1.000000
